# Nagel–Schreckenberg (16 points)

Nagel–Schreckenberg  is a simple model used for modelling motorway traffic. As the name suggest, it was developed by two German physicists Kai Nagel and Michael Schreckenberg in the early 1990s.
## Road
The road is divided into cells. Each cell represents a part of the road. A cell can be either empty or contains a single car (only one car in any cell).
## Car 
Cars are one cell long and have a velocity $v$, originally in range of 0-5 units per time unit.
## Transisition rules
In the transition function the following steps are performed (mind the order):
* **Acceleration**: The velocity of all cars having a velocity lower than the maximum velocity is increased by 1.
* **Slowing down**: If the distance between the current and the following car is smaller than the car's velocity, the velocity is reduced to the number of empty cells in front of the car (avoiding collisions).
* **Randomization**: The speed of all cars that have a $v > 1$ is reduced by one unit with the probability of *p*. 
* **Car motion**: All cars are moved *v* cells ahead.


Discretisation of time and space results in cellular automata. If the maximum speed is equal to 1, the model is identical to elementary cellular automaton with rule 184. 
(source and more detailed description: [Model](https://en.wikipedia.org/wiki/Nagel–Schreckenberg_model) and [184](https://en.wikipedia.org/wiki/Rule_184))

## Basic implementation of Nagel-Schreckenberg model (8 points)

* (5 points) Create a working Nagel-Schreckenberg model using the following template, fill in the gaps. Start with randomly generate state.
* (2 points) Add periodic boundaries, i.e. cars leaving the road should appear at the beginning e.g. a car with $v=5$ located 2 cells before the end of the road should be moved to the third cell of the road.
* (1 point) Add a function that will randomly add new cars at the first cell of the road. Remember to check if the cell is empty, spawn a car only if possible. 

In [38]:
import numpy as np
import itertools
import pygame
import random
import copy

from pygame.locals import (
    K_UP,
    K_DOWN,
    K_LEFT,
    K_RIGHT,
    K_ESCAPE,
    KEYDOWN,
    QUIT,
)

In [39]:
class Car:
    
    def __init__(self, minVelocity, maxVelocity, position):
        # assign a randomly selected colour for visualisation
        self.colour = (random.randint(0,255), random.randint(0,255), random.randint(0,255))
        
        # copy parameters
        self.minVelocity = minVelocity
        self.maxVelocity = maxVelocity
        
        # generate random init velocity
        self.velocity = random.randint(minVelocity,maxVelocity)
                
        # set initial position
        self.position = position
    
    def increaseVelocity(self):
        if self.velocity < self.maxVelocity:
            self.velocity = self.velocity + 1
    
    def setVelocity(self, newVelocity):
        if newVelocity <= self.maxVelocity:
            self.velocity = newVelocity
        
    def randomizeVelocity(self, probability):
        if self.velocity > 1 and random.random() < probability:
            self.velocity -= 1


In [40]:
class Model:
    def __init__(self):
        # Parameters
        self.roadLength = 50
        self.minVelocity = 0
        self.maxVelocity = 5
        self.probabilityOfNewCar = 0.87
        initalNumberOfCars = 5
        
        self.time = 0
        self.states = []
        
        # Generate cars 
        initalPositions = random.sample(range(0, self.roadLength), initalNumberOfCars)
        self.cars = []
        for i in initalPositions:
            self.cars.append(Car(self.minVelocity, self.maxVelocity, i))
            
        # Sort cars arrays
        self.cars.sort(key = lambda x: x.position)

        self.car_img = pygame.image.load('car-icon.png')
    
    def randomlyAddNewCar(self):
        if random.random() < self.probabilityOfNewCar:
            if self.road[0] is None:  # Check if the first cell is empty
                new_car = Car(self.minVelocity, self.maxVelocity, 0)
                self.cars.append(new_car)
                self.road[0] = new_car
    
    def update(self):
        
        # 0. Save previous state
        self.states.append(copy.deepcopy(self.cars))
        self.time += 1

        # I. Acceleration
        for car in self.cars:
            car.increaseVelocity()

        # II. Slowing Down
        for i in range(len(self.cars)):
            car = self.cars[i]
            next_car = self.cars[(i + 1) % len(self.cars)]
            distance = (next_car.position - car.position - 1) % self.roadLength
            if distance < car.velocity:
                car.setVelocity(distance)
                
        # III. Randomization
        for car in self.cars:
            car.randomizeVelocity(probability=0.3)

        # IV. Car motion
        self.road = [None] * self.roadLength
        for car in self.cars:
            car.position = (car.position + car.velocity) % self.roadLength
            self.road[car.position] = car


        # To simplify calculations we're sorting the cars by their position
        self.randomlyAddNewCar()
        self.cars.sort(key = lambda x: x.position)
        

    def stepBack(self):
        if len(self.states) > 0:
            self.cars = self.states.pop()
            self.time -= 1
        
    def draw(self, screen, w_width, w_height):
        black = (0,0,0)
        white = (255,255,255)  
        grey = (128,128,128)
        blockSize = w_width / self.roadLength

        # draw road
        pos_x = 0
        pos_y = w_height/2
        rect = pygame.Rect(pos_x, pos_y, (blockSize)*self.roadLength, blockSize)
        pygame.draw.rect(screen, grey, rect, 0)

    
        for car in self.cars:
            pos_x = (blockSize) * car.position
            pos_y = w_height/2
            rect = pygame.Rect(pos_x, pos_y, blockSize, blockSize)
            # Draw Cars
            pygame.draw.rect(screen, car.colour, rect, 0)
            screen.blit(pygame.transform.scale(self.car_img, (blockSize, blockSize)), rect)

        pygame.display.flip()

In [41]:
pygame.init()
w_width = 1200
w_height = 800
screen = pygame.display.set_mode([w_width, w_height])
model = Model()
screen.fill((255, 255, 255))
running = True
model.draw(screen, w_width, w_height)
while running:
    for event in pygame.event.get():   
        if event.type == QUIT:
            running = False
        
        if event.type == KEYDOWN:
            if event.key == K_RIGHT:
                model.update()
                model.draw(screen, w_width, w_height)
            elif event.key == K_LEFT:
                model.stepBack()
                model.draw(screen, w_width, w_height)
        
pygame.quit()

## Improvements (8 points)
Add:
* (1 points) Labels indicating time and number of cars.
* (2 points) Plot showing average speed vs car density (amount of cars per cell; note that it should be $<1$).
* (5 points) Create two lane motorway, implement overtaking. Car changes the lane if $v>distance$ and the left/right lane is empty (remember to check for collisions  with cars already driving the left/right lane). 
To get this points you must do a full implmenetation of basic model (including periodic boundaries and random generation). 

In [42]:
class AdvancedCar(Car):
    def __init__(self, minVelocity, maxVelocity, position, lane=0):
        super().__init__(minVelocity, maxVelocity, position)
        self.lane = lane  # lane: 0 for right lane, 1 for left lane
    
    def change_lane(self, new_lane):
        self.lane = new_lane


In [45]:
class AdvancedModel(Model):
    def __init__(self):
        super().__init__()
        
        self.lane_count = 2
        self.road = [[None] * self.roadLength for _ in range(self.lane_count)]

        # Initialize cars in a random lane
        for car in self.cars:
            car.__class__ = AdvancedCar  # Upgrade existing Car objects to AdvancedCar
            car.lane = random.choice([i for i in range(self.lane_count)])  # Randomly assign a lane

    def lane_empty(self, car, new_lane):
        """Check if the target lane is empty for the car's movement to avoid collision"""
        next_position = (car.position + car.velocity) % self.roadLength
        return self.road[new_lane][next_position] is None

    def update(self):
        # Save previous state
        self.states.append(copy.deepcopy(self.cars))
        self.time += 1

        # Step I: Acceleration
        for car in self.cars:
            car.increaseVelocity()

        # Step II: Slowing Down
        for i in range(len(self.cars)):
            car = self.cars[i]
            next_car = self.cars[(i + 1) % len(self.cars)]
            distance = (next_car.position - car.position - 1) % self.roadLength
            if distance < car.velocity:
                car.setVelocity(distance)

        # Step III: Randomization
        for car in self.cars:
            car.randomizeVelocity(probability=0.3)

        # Step IV: Overtaking - Check for lane change opportunities
        for car in self.cars:
            if car.velocity > distance:  # Condition to consider overtaking
                new_lane = (car.lane + 1) % self.lane_count  # Attempt to change lane
                if self.lane_empty(car, new_lane):
                    car.change_lane(new_lane)  # Change lane if clear

        # Step V: Car motion and road update
        self.road = [[None] * self.roadLength for _ in range(self.lane_count)]
        for car in self.cars:
            car.position = (car.position + car.velocity) % self.roadLength
            self.road[car.lane][car.position] = car  # Update car's new position in the lane

        # Optionally add new car
        self.randomlyAddNewCar()

        # Sort cars by position after all updates
        self.cars.sort(key=lambda x: x.position)

    def randomlyAddNewCar(self):
        if random.random() < self.probabilityOfNewCar:
            for lane_num, lane in enumerate(self.road):
                if lane[0] is None:  # Check if the first cell is empty
                    new_car = AdvancedCar(self.minVelocity, self.maxVelocity, 0, lane_num)
                    self.cars.append(new_car)
                    self.road[lane_num][0] = new_car
                    return # if added dont add to another line

    def draw(self, screen, w_width, w_height):
        black = (0, 0, 0)
        grey = (128, 128, 128)
        font_color = (255, 255, 255)
        
        # Road dimensions for two lanes
        block_size = w_width / self.roadLength
        font = pygame.font.SysFont(None, 36)

        # Clear screen
        screen.fill(black)

        # Draw two lanes
        for lane in range(self.lane_count):
            pos_y = (w_height / 2) + ((lane - 0.5) * block_size)
            rect = pygame.Rect(0, pos_y, w_width, block_size)
            pygame.draw.rect(screen, grey, rect, 0)

        # Draw cars on both lanes
        for car in self.cars:
            pos_x = block_size * car.position
            pos_y = (w_height / 2) + (car.lane - 0.5) * block_size
            rect = pygame.Rect(pos_x, pos_y, block_size, block_size)
            pygame.draw.rect(screen, car.colour, rect, 0)
            screen.blit(pygame.transform.scale(self.car_img, (block_size, block_size)), rect)  # Car icon

        # Draw labels for time and car count
        time_text = font.render(f"Time: {self.time}", True, font_color)
        car_count_text = font.render(f"Cars: {len(self.cars)}", True, font_color)
        screen.blit(time_text, (10, 10))
        screen.blit(car_count_text, (10, 40))

        pygame.display.flip()


In [46]:
pygame.init()
w_width = 1200
w_height = 800
screen = pygame.display.set_mode([w_width, w_height])
model = AdvancedModel()
screen.fill((255, 255, 255))

    
running = True
model.draw(screen, w_width, w_height)
while running:
    for event in pygame.event.get():   
        if event.type == QUIT:
            running = False
        
        if event.type == KEYDOWN:
            if event.key == K_RIGHT:
                model.update()
                model.draw(screen, w_width, w_height)
            elif event.key == K_LEFT:
                model.stepBack()
                model.draw(screen, w_width, w_height)


        
pygame.quit()

data = model.getPlotData()

AttributeError: 'AdvancedModel' object has no attribute 'getPlotData'